In [109]:
# Configurando as variáveis de ambiente

import os 
os.environ["SPARK_HOME"] = "C:\spark\spark-3.1.3-bin-hadoop2.7"

In [110]:
# Tornar o pyspark "importável"
import findspark
findspark.init()

In [111]:
# Iniciando uma sessão local

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master('local[*]').getOrCreate()

In [112]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.functions import col

In [113]:
spark

In [114]:
# Lendo o arquivo infomix
path = 'Desktop\data_etl_test (1)\infomix.tsv'
df_infomix = spark.read.csv(path, sep = '\t', header=True)

In [115]:
# Contando as linhas do DataFrame
df_infomix.count()

25844

In [116]:
df_infomix.limit(10).toPandas()

,cnpj,gtin,category
0,08540603000198,06890101502534,massas e molhos
1,71322150004408,07898605250417,cervejas
2,43235985000732,07896000724465,cabelo
3,47508411134004,07896102589122,conservas e enlatados
4,47508411135833,07895000474646,bomboniere
5,09379531000101,05893202502558,massas e molhos
6,47508411135833,07891008188271,bomboniere
7,39346861026802,07891025112983,iogurtes e sobremesas
8,47508411134004,07896262304368,bomboniere
9,41185455001501,07896085393488,leite


In [117]:
df_infomix.printSchema()

root
 |-- cnpj: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- category: string (nullable = true)



In [118]:
# Lendo o arquivo descricoes_externas
path = 'Desktop\data_etl_test (1)\descricoes_externas.tsv'
df_descricoes_externas = spark.read.csv(path, sep = '\t', header=True)

In [119]:
df_descricoes_externas.count()

57203

In [120]:
df_descricoes_externas.limit(10).toPandas()

,id,gtin,description,flag_infoprice
0,2482686,07896512911933,SABONETE INFANTIL LIQUIDO GRANADO BEBE LAVANDA...,true
1,2482685,07896512911933,SAB GRANADO BEBE LAVANDA 250G,false
2,2482684,07896512911933,SAB LIQ.GRAN.BEBE LAVAN 25,false
3,2482683,07896512911933,SAB GRANADO GLICLAVAN BEBE 250M,false
4,2482682,07896512911933,SAB BABY GRA LA 250ML,false
5,2482681,07896512911933,SABONETE LIQ GRANADO 250ML LAV,false
6,2057830,07896005286579,MACARRAO COM OVOS ESPAGUETE N8 DONA BENTA 500G,true
7,2057829,07896005286579,M.D.BENTA ESPAGUETE N8,false
8,2057828,07896005286579,MAC ESPAG DONA BENTA C OVOS N8 500GR,false
9,2057827,07896005286579,ESPAGUETE DONA BENTA COM OVOS,false


In [121]:
# Lendo o arquivo cosmos
cosmos = spark.read.format('json').load('Desktop\data_etl_test (1)\cosmos.jl', multiLine = "False")

In [122]:
cosmos.count()

20588

In [123]:
cosmos.limit(10).toPandas()

,created_at,gtin,id,response
0,2018-10-22 19:24:28,07896040700276,16bde255-988c-466b-9d35-39b706b9fbda,"(, None, LAVA ROUPAS COQUETEL ERVAS, (10000424..."
1,2018-10-20 00:37:14,00000078909434,96f61c2b-5057-439c-9717-847efe5ea7a5,"(AUDITADO, FERRERO ROCHER, BOMBOM FERRERO ROCH..."
2,2018-10-25 04:44:33,07898917945100,8ee42cd3-4200-45dc-a10f-6b9e62bc5127,"(None, None, None, None, None, None, None, GTI..."
3,2018-10-23 05:23:08,27702018072396,bda1d82f-cfad-482c-a243-65ae7f04a7bd,"(None, None, None, None, None, None, None, GTI..."
4,2019-02-07 21:53:39,07896005279595,427b4261-d73d-4c13-a085-d9d5bb988c39,"(None, None, None, None, None, None, None, GTI..."
5,2018-10-22 19:32:16,07896221600036,826540b0-f18e-4491-903a-05800a021656,"(NÃO AUDITADO, None, AGUA SANITARIA DRAGAO 2L,..."
6,2018-10-21 16:09:29,07891025113157,5a1bf587-0362-468e-82f8-6f18b10cd876,"(None, None, None, None, None, None, None, GTI..."
7,2018-10-21 15:25:41,07896259412885,286a0add-c333-4b29-8fbb-3be273e758c0,"(None, None, None, None, None, None, None, GTI..."
8,2018-10-21 15:24:02,07891150057937,097235d0-51e5-4c81-a504-a23fee2f1e9e,"(None, None, None, None, None, None, None, GTI..."
9,2018-10-23 03:08:26,00891454037366,629e26c3-2c08-40f7-b63e-eaf3809121e6,"(None, None, None, None, None, None, None, GTI..."


In [124]:
cosmos.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response: struct (nullable = true)
 |    |-- autited: string (nullable = true)
 |    |-- brand: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- gcp: struct (nullable = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |-- gtin: string (nullable = true)
 |    |-- image: string (nullable = true)
 |    |-- ncm: struct (nullable = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- units: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- gtin: string (nullable = true)
 |    |    |    |-- packing_size: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)



In [125]:
cosmos2 =( cosmos.select(
        cosmos['created_at'], 
        col('gtin').alias('gtin1'), 
        cosmos['id'], 
        col('response.*'))      
         )

In [126]:
cosmos2.limit(2).toPandas()

,created_at,gtin1,id,autited,brand,description,gcp,gtin,image,ncm,status,units
0,2018-10-22 19:24:28,07896040700276,16bde255-988c-466b-9d35-39b706b9fbda,,None,LAVA ROUPAS COQUETEL ERVAS,"(10000424, Detergentes para a roupa)",07896040700276,https://cdn-cosmos.bluesoft.com.br/products/78...,"(3402.20.00, Preparações acondicionadas para v...",OK,"[(7896040700276, 1, Unidade)]"
1,2018-10-20 00:37:14,00000078909434,96f61c2b-5057-439c-9717-847efe5ea7a5,AUDITADO,FERRERO ROCHER,BOMBOM FERRERO ROCHER 3X16G,"(None, None)",00000078909434,https://cdn-cosmos.bluesoft.com.br/products/78...,"(1806.90.00, Outros)",OK,"[(78909434, 1, Unidade)]"


In [127]:
cosmos2.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- gtin1: string (nullable = true)
 |-- id: string (nullable = true)
 |-- autited: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description: string (nullable = true)
 |-- gcp: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- description: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- image: string (nullable = true)
 |-- ncm: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- description: string (nullable = true)
 |-- status: string (nullable = true)
 |-- units: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- gtin: string (nullable = true)
 |    |    |-- packing_size: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [128]:
cosmos3 = (cosmos2.select(
        cosmos2['created_at'], \
        cosmos2['gtin1'], \
        cosmos2['id'], \
        cosmos2['autited'], \
        cosmos2['brand'], \
        col('description').alias('description1'), \
        col('gcp.*'), \
        cosmos2['gtin'], \
        cosmos2['image'], \
        cosmos2['ncm'], \
        cosmos2['status'], \
        cosmos2['units']))

In [129]:
cosmos3.limit(10).toPandas()

,created_at,gtin1,id,autited,brand,description1,code,description,gtin,image,ncm,status,units
0,2018-10-22 19:24:28,07896040700276,16bde255-988c-466b-9d35-39b706b9fbda,,None,LAVA ROUPAS COQUETEL ERVAS,10000424,Detergentes para a roupa,07896040700276,https://cdn-cosmos.bluesoft.com.br/products/78...,"(3402.20.00, Preparações acondicionadas para v...",OK,"[(7896040700276, 1, Unidade)]"
1,2018-10-20 00:37:14,00000078909434,96f61c2b-5057-439c-9717-847efe5ea7a5,AUDITADO,FERRERO ROCHER,BOMBOM FERRERO ROCHER 3X16G,None,None,00000078909434,https://cdn-cosmos.bluesoft.com.br/products/78...,"(1806.90.00, Outros)",OK,"[(78909434, 1, Unidade)]"
2,2018-10-25 04:44:33,07898917945100,8ee42cd3-4200-45dc-a10f-6b9e62bc5127,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
3,2018-10-23 05:23:08,27702018072396,bda1d82f-cfad-482c-a243-65ae7f04a7bd,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
4,2019-02-07 21:53:39,07896005279595,427b4261-d73d-4c13-a085-d9d5bb988c39,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
5,2018-10-22 19:32:16,07896221600036,826540b0-f18e-4491-903a-05800a021656,NÃO AUDITADO,None,AGUA SANITARIA DRAGAO 2L,None,None,07896221600036,/assets/product-placeholder-55580b779c0ed40855...,"(None, None)",OK,"[(7896221600036, 1, Unidade)]"
6,2018-10-21 16:09:29,07891025113157,5a1bf587-0362-468e-82f8-6f18b10cd876,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
7,2018-10-21 15:25:41,07896259412885,286a0add-c333-4b29-8fbb-3be273e758c0,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
8,2018-10-21 15:24:02,07891150057937,097235d0-51e5-4c81-a504-a23fee2f1e9e,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
9,2018-10-23 03:08:26,00891454037366,629e26c3-2c08-40f7-b63e-eaf3809121e6,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None


In [130]:
cosmos3.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- gtin1: string (nullable = true)
 |-- id: string (nullable = true)
 |-- autited: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- image: string (nullable = true)
 |-- ncm: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- description: string (nullable = true)
 |-- status: string (nullable = true)
 |-- units: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- gtin: string (nullable = true)
 |    |    |-- packing_size: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [131]:
cosmos4 = (cosmos3.select(
           cosmos3['created_at'], \
           cosmos3['gtin1'], \
           cosmos3['id'], \
           cosmos3['autited'], \
           cosmos3['brand'], \
           cosmos3['description1'], \
           cosmos3['description'],
           col('code').alias('gcp_code'),\
           col('description').alias('gcp_description'), \
           cosmos3['gtin'], \
           cosmos3['image'], \
           cosmos3['ncm'], \
           cosmos3['status'],
           cosmos3['units'])
          )

In [132]:
cosmos4.limit(5).toPandas()

,created_at,gtin1,id,autited,brand,description1,description,gcp_code,gcp_description,gtin,image,ncm,status,units
0,2018-10-22 19:24:28,07896040700276,16bde255-988c-466b-9d35-39b706b9fbda,,None,LAVA ROUPAS COQUETEL ERVAS,Detergentes para a roupa,10000424,Detergentes para a roupa,07896040700276,https://cdn-cosmos.bluesoft.com.br/products/78...,"(3402.20.00, Preparações acondicionadas para v...",OK,"[(7896040700276, 1, Unidade)]"
1,2018-10-20 00:37:14,00000078909434,96f61c2b-5057-439c-9717-847efe5ea7a5,AUDITADO,FERRERO ROCHER,BOMBOM FERRERO ROCHER 3X16G,None,None,None,00000078909434,https://cdn-cosmos.bluesoft.com.br/products/78...,"(1806.90.00, Outros)",OK,"[(78909434, 1, Unidade)]"
2,2018-10-25 04:44:33,07898917945100,8ee42cd3-4200-45dc-a10f-6b9e62bc5127,None,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
3,2018-10-23 05:23:08,27702018072396,bda1d82f-cfad-482c-a243-65ae7f04a7bd,None,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None
4,2019-02-07 21:53:39,07896005279595,427b4261-d73d-4c13-a085-d9d5bb988c39,None,None,None,None,None,None,None,None,None,GTIN_NOT_FOUND,None


In [133]:
cosmos4.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- gtin1: string (nullable = true)
 |-- id: string (nullable = true)
 |-- autited: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- description: string (nullable = true)
 |-- gcp_code: string (nullable = true)
 |-- gcp_description: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- image: string (nullable = true)
 |-- ncm: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- description: string (nullable = true)
 |-- status: string (nullable = true)
 |-- units: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- gtin: string (nullable = true)
 |    |    |-- packing_size: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [134]:
cosmos5 = (cosmos4.select(
           cosmos4['created_at'], \
           cosmos4['gtin1'], \
           cosmos4['id'], \
           cosmos4['autited'], \
           cosmos4['brand'], \
           cosmos4['description1'], \
           cosmos4['gcp_code'], \
           cosmos4['gcp_description'], \
           col('gtin').alias('gtin2'), \
           cosmos4['image'], \
           cosmos4['status'],\
           cosmos4['units'])
          )

In [135]:
cosmos5.limit(2).toPandas()

,created_at,gtin1,id,autited,brand,description1,gcp_code,gcp_description,gtin2,image,status,units
0,2018-10-22 19:24:28,07896040700276,16bde255-988c-466b-9d35-39b706b9fbda,,None,LAVA ROUPAS COQUETEL ERVAS,10000424,Detergentes para a roupa,07896040700276,https://cdn-cosmos.bluesoft.com.br/products/78...,OK,"[(7896040700276, 1, Unidade)]"
1,2018-10-20 00:37:14,00000078909434,96f61c2b-5057-439c-9717-847efe5ea7a5,AUDITADO,FERRERO ROCHER,BOMBOM FERRERO ROCHER 3X16G,None,None,00000078909434,https://cdn-cosmos.bluesoft.com.br/products/78...,OK,"[(78909434, 1, Unidade)]"


In [136]:
cosmos5.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- gtin1: string (nullable = true)
 |-- id: string (nullable = true)
 |-- autited: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- gcp_code: string (nullable = true)
 |-- gcp_description: string (nullable = true)
 |-- gtin2: string (nullable = true)
 |-- image: string (nullable = true)
 |-- status: string (nullable = true)
 |-- units: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- gtin: string (nullable = true)
 |    |    |-- packing_size: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [137]:
df_cosmos = (cosmos5.select(
           col('gtin1').alias('gtin'), 
           cosmos5['gtin2'], 
           cosmos5['brand'], 
           col('description1').alias('description'), 
           col('gcp_description').alias('category_description'), 
           cosmos5['status'])
          )

In [138]:
df_cosmos.count()

20588

In [139]:
df_cosmos.limit(5).toPandas()

,gtin,gtin2,brand,description,category_description,status
0,07896040700276,07896040700276,None,LAVA ROUPAS COQUETEL ERVAS,Detergentes para a roupa,OK
1,00000078909434,00000078909434,FERRERO ROCHER,BOMBOM FERRERO ROCHER 3X16G,None,OK
2,07898917945100,None,None,None,None,GTIN_NOT_FOUND
3,27702018072396,None,None,None,None,GTIN_NOT_FOUND
4,07896005279595,None,None,None,None,GTIN_NOT_FOUND


In [140]:
df_cosmos.printSchema()

root
 |-- gtin: string (nullable = true)
 |-- gtin2: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category_description: string (nullable = true)
 |-- status: string (nullable = true)



In [141]:
# Lendo o arquivo gs1.jl e inserindo em um DataFrame
gs = spark.read.format('json').load('Desktop\data_etl_test (1)\gs1.jl',\
                                                       multiLine = "False")

In [142]:
# Contando as linhas do arquivo
gs.count()

19051

In [143]:
# Lendo os 2 primeiros registros
gs.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,"(None, ((Row(city='SAO PAULO', cityCode='', co..."
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,"(None, ((Row(city='CAMPINAS', cityCode='', cou..."


In [144]:
# Visualizando o schema do DataFrame
gs.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response: struct (nullable = true)
 |    |-- error: struct (nullable = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- extendedHelp: string (nullable = true)
 |    |    |-- message: string (nullable = true)
 |    |-- gepirParty: struct (nullable = true)
 |    |    |-- partyDataLine: struct (nullable = true)
 |    |    |    |-- address: struct (nullable = true)
 |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |-- cityCode: string (nullable = true)
 |    |    |    |    |-- countryCode: struct (nullable = true)
 |    |    |    |    |    |-- _: string (nullable = true)
 |    |    |    |    |    |-- codeListVersion: string (nullable = true)
 |    |    |    |    |-- countyCode: string (nullable = true)
 |    |    |    |    |-- crossStreet: string (nullable = true)
 |    |    | 

In [145]:
# Expandindo a coluna response
gs1 =( gs.select(
        gs['cnpj_manufacturer'], 
        gs['created_at'], 
        gs['gtin'], 
        gs['id'],        
        col('response.*')) 
        )

In [146]:
# Lendo os 2 primeiros registros
gs1.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,error,gepirParty,responseHeader,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,"(((SAO PAULO, , Row(_='', codeListVersion='1')...","((0,), 1, 9501101020016)",OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,"(((CAMPINAS, , Row(_='', codeListVersion='1'),...","((0,), 1, 9501101020016)",OK


In [147]:
# Visualizando o schema do DataFrame
gs1.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- error: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- extendedHelp: string (nullable = true)
 |    |-- message: string (nullable = true)
 |-- gepirParty: struct (nullable = true)
 |    |-- partyDataLine: struct (nullable = true)
 |    |    |-- address: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- cityCode: string (nullable = true)
 |    |    |    |-- countryCode: struct (nullable = true)
 |    |    |    |    |-- _: string (nullable = true)
 |    |    |    |    |-- codeListVersion: string (nullable = true)
 |    |    |    |-- countyCode: string (nullable = true)
 |    |    |    |-- crossStreet: string (nullable = true)
 |    |    |    |-- currencyOfPartyCode: struct (nullable = true)
 |    |    |    |    |-- _: string (nullable = true)
 |  

In [148]:
# Renomeando as colunas expandidas com o alias da coluna anterior
gs2 =( gs1.select(
        gs1['cnpj_manufacturer'], 
        gs1['created_at'], 
        gs1['gtin'], 
        gs1['id'],
        col('error').alias('response-error'),
        gs1['gepirParty'],
        gs1['responseHeader'],
        gs1['status'])      
        )

In [149]:
# Visualizando os 2 primeiros registros
gs2.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response-error,gepirParty,responseHeader,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,"(((SAO PAULO, , Row(_='', codeListVersion='1')...","((0,), 1, 9501101020016)",OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,"(((CAMPINAS, , Row(_='', codeListVersion='1'),...","((0,), 1, 9501101020016)",OK


In [150]:
# Expandindo as colunas response-error, gepirParty e responseHeader
gs3 =( gs2.select(
        gs2['cnpj_manufacturer'], 
        gs2['created_at'], 
        gs2['gtin'], 
        gs2['id'],
        col('response-error.*'),
        col('gepirParty.*'),
        col('responseHeader.*'),
        gs2['status'])      
        )

In [151]:
# Lendo os 2 primeiros registros
gs3.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,code,extendedHelp,message,partyDataLine,gepirReturnCode,numberOfHits,responderGLN,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,None,None,"((SAO PAULO, , (, 1), , , (, 1), (, ), (, 1), ...","(0,)",1,9501101020016,OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,None,None,"((CAMPINAS, , (, 1), , , (, 1), (, ), (, 1), A...","(0,)",1,9501101020016,OK


In [152]:
# Renomeando as colunas expandidas com o alias da coluna anterior
gs4 =( gs3.select(
        gs3['cnpj_manufacturer'], 
        gs3['created_at'], 
        gs3['gtin'], 
        gs3['id'],
        col('code').alias('response-error-code'),
        col('extendedHelp').alias('response-error-extendedHelp'),
        col('message').alias('response-error-message'),
        col('partyDataLine').alias('gepirParty-partyDataLine'),
        col('gepirReturnCode').alias('responseHeader-gepirReturnCode'),
        col('numberOfHits').alias('responseHeader-numberOfHits'),
        col('responderGLN').alias('responseHeader-responderGLN'),
        gs3['status'])      
        )

In [153]:
gs4.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response-error-code,response-error-extendedHelp,response-error-message,gepirParty-partyDataLine,responseHeader-gepirReturnCode,responseHeader-numberOfHits,responseHeader-responderGLN,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,None,None,"((SAO PAULO, , (, 1), , , (, 1), (, ), (, 1), ...","(0,)",1,9501101020016,OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,None,None,"((CAMPINAS, , (, 1), , , (, 1), (, ), (, 1), A...","(0,)",1,9501101020016,OK


In [154]:
gs4.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response-error-code: string (nullable = true)
 |-- response-error-extendedHelp: string (nullable = true)
 |-- response-error-message: string (nullable = true)
 |-- gepirParty-partyDataLine: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- cityCode: string (nullable = true)
 |    |    |-- countryCode: struct (nullable = true)
 |    |    |    |-- _: string (nullable = true)
 |    |    |    |-- codeListVersion: string (nullable = true)
 |    |    |-- countyCode: string (nullable = true)
 |    |    |-- crossStreet: string (nullable = true)
 |    |    |-- currencyOfPartyCode: struct (nullable = true)
 |    |    |    |-- _: string (nullable = true)
 |    |    |    |-- codeListVersion: string (nullable = true)
 |    |    |-- geographicalCoordina

In [155]:
# Expandindo as colunas gepirParty-partyDataLine e responseHeader-gepirReturnCode
gs5 =( gs4.select(
        gs4['cnpj_manufacturer'], 
        gs4['created_at'], 
        gs4['gtin'], 
        gs4['id'],
        gs4['response-error-code'],
        gs4['response-error-extendedHelp'],
        gs4['response-error-message'],
        col('gepirParty-partyDataLine.*'),
        col('responseHeader-gepirReturnCode.*'),
        gs4['responseHeader-numberOfHits'],
        gs4['responseHeader-responderGLN'],
        gs4['status'])
        )

In [156]:
# Lendo os 2 primeiros registros
gs5.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response-error-code,response-error-extendedHelp,response-error-message,address,contact,countryAdministered,...,gepirRequestedKey,informationProvider,lastChangeDate,partyDataLanguage,responderSpecificData,returnCode,_,responseHeader-numberOfHits,responseHeader-responderGLN,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,None,None,"(SAO PAULO, , (, 1), , , (, 1), (, ), (, 1), N...",None,BR,...,"((GTIN, None), 07891000258491, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,None,None,"(CAMPINAS, , (, 1), , , (, 1), (, ), (, 1), AR...",None,BR,...,"((GTIN, None), 07898142863118, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK


In [157]:
# Visualizando o schema do DataFrame
gs5.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response-error-code: string (nullable = true)
 |-- response-error-extendedHelp: string (nullable = true)
 |-- response-error-message: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- cityCode: string (nullable = true)
 |    |-- countryCode: struct (nullable = true)
 |    |    |-- _: string (nullable = true)
 |    |    |-- codeListVersion: string (nullable = true)
 |    |-- countyCode: string (nullable = true)
 |    |-- crossStreet: string (nullable = true)
 |    |-- currencyOfPartyCode: struct (nullable = true)
 |    |    |-- _: string (nullable = true)
 |    |    |-- codeListVersion: string (nullable = true)
 |    |-- geographicalCoordinates: struct (nullable = true)
 |    |    |-- latitude: string (nullable = true)
 |    |    |-- longitude: string (nu

In [158]:
# Renomeando as colunas expandidas com o alias da coluna anterior
gs6 =( gs5.select(
        gs5['cnpj_manufacturer'], 
        gs5['created_at'], 
        gs5['gtin'], 
        gs5['id'],
        gs5['response-error-code'],
        gs5['response-error-extendedHelp'],
        gs5['response-error-message'],
        col('address').alias('gepirParty-partyDataLine-address'),
        col('contact').alias('gepirParty_partyDataLine_contact'),
        col('countryAdministered').alias('gepirParty-partyDataLine-countryAdministered'),
        col('gS1CompanyPrefix').alias('gepirParty-partyDataLine-gS1CompanyPrefix'),
        col('gS1CompanyPrefixLicensee').alias('gepirParty-partyDataLine-gS1CompanyPrefixLicensee'),
        col('gS1KeyLicensee').alias('gepirParty-partyDataLine-gS1KeyLicensee'),
        col('gepirItemExternalFileLink').alias('gepirParty-partyDataLine-gepirItemExternalFileLink'),
        col('gepirRequestedKey').alias('gepirParty-partyDataLine-gepirRequestedKey'),
        col('informationProvider').alias('gepirParty-partyDataLine-informationProvider'),
        col('lastChangeDate').alias('gepirParty-partyDataLine-lastChangeDate'),
        col('partyDataLanguage').alias('gepirParty-partyDataLine-partyDataLanguage'),
        col('responderSpecificData').alias('gepirParty-partyDataLine-responderSpecificData'),
        col('returnCode').alias('gepirParty-partyDataLine-returnCode'),
        col('_').alias('responseHeader-gepirReturnCode-_'),
        gs5['responseHeader-numberOfHits'],
        gs5['responseHeader-responderGLN'],
        gs5['status'])
        )

In [159]:
# Lendo os 2 primeiros registros
gs6.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response-error-code,response-error-extendedHelp,response-error-message,gepirParty-partyDataLine-address,gepirParty_partyDataLine_contact,gepirParty-partyDataLine-countryAdministered,...,gepirParty-partyDataLine-gepirRequestedKey,gepirParty-partyDataLine-informationProvider,gepirParty-partyDataLine-lastChangeDate,gepirParty-partyDataLine-partyDataLanguage,gepirParty-partyDataLine-responderSpecificData,gepirParty-partyDataLine-returnCode,responseHeader-gepirReturnCode-_,responseHeader-numberOfHits,responseHeader-responderGLN,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,None,None,"(SAO PAULO, , (, 1), , , (, 1), (, ), (, 1), N...",None,BR,...,"((GTIN, None), 07891000258491, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,None,None,"(CAMPINAS, , (, 1), , , (, 1), (, ), (, 1), AR...",None,BR,...,"((GTIN, None), 07898142863118, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK


In [160]:
# Visualizando o schema do DataFrame
gs6.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response-error-code: string (nullable = true)
 |-- response-error-extendedHelp: string (nullable = true)
 |-- response-error-message: string (nullable = true)
 |-- gepirParty-partyDataLine-address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- cityCode: string (nullable = true)
 |    |-- countryCode: struct (nullable = true)
 |    |    |-- _: string (nullable = true)
 |    |    |-- codeListVersion: string (nullable = true)
 |    |-- countyCode: string (nullable = true)
 |    |-- crossStreet: string (nullable = true)
 |    |-- currencyOfPartyCode: struct (nullable = true)
 |    |    |-- _: string (nullable = true)
 |    |    |-- codeListVersion: string (nullable = true)
 |    |-- geographicalCoordinates: struct (nullable = true)
 |    |    |-- latitude: string (nullable = true)
 |    |    

In [161]:
# Expandindo as colunas gepirParty-partyDataLine-address, gepirParty_partyDataLine_contact

gs7 =( gs6.select(
        gs6['cnpj_manufacturer'], 
        gs6['created_at'], 
        gs6['gtin'], 
        gs6['id'],
        gs6['response-error-code'],
        gs6['response-error-extendedHelp'],
        gs6['response-error-message'],
        col('gepirParty-partyDataLine-address.*'),
        col('gepirParty_partyDataLine_contact.*'),
        gs6['gepirParty-partyDataLine-countryAdministered'],
        gs6['gepirParty-partyDataLine-gS1CompanyPrefix'],
        col('gepirParty-partyDataLine-gS1CompanyPrefixLicensee.*'),
        gs6['gepirParty-partyDataLine-gS1KeyLicensee'],
        gs6['gepirParty-partyDataLine-gepirItemExternalFileLink'],
        gs6['gepirParty-partyDataLine-gepirRequestedKey'],
        gs6['gepirParty-partyDataLine-informationProvider'],
        gs6['gepirParty-partyDataLine-lastChangeDate'],
        gs6['gepirParty-partyDataLine-partyDataLanguage'],
        gs6['gepirParty-partyDataLine-responderSpecificData'],
        gs6['gepirParty-partyDataLine-returnCode'],
        gs6['responseHeader-gepirReturnCode-_'],
        gs6['responseHeader-numberOfHits'],
        gs6['responseHeader-responderGLN'],
        gs6['status'])
     )

In [162]:
# Visualizando os dois primeiros registros
gs7.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response-error-code,response-error-extendedHelp,response-error-message,city,cityCode,countryCode,...,gepirParty-partyDataLine-gepirRequestedKey,gepirParty-partyDataLine-informationProvider,gepirParty-partyDataLine-lastChangeDate,gepirParty-partyDataLine-partyDataLanguage,gepirParty-partyDataLine-responderSpecificData,gepirParty-partyDataLine-returnCode,responseHeader-gepirReturnCode-_,responseHeader-numberOfHits,responseHeader-responderGLN,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,None,None,SAO PAULO,,"(, 1)",...,"((GTIN, None), 07891000258491, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,None,None,CAMPINAS,,"(, 1)",...,"((GTIN, None), 07898142863118, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK


In [163]:
# Lendo os 2 primeiros registros
gs7.limit(2).toPandas()

,cnpj_manufacturer,created_at,gtin,id,response-error-code,response-error-extendedHelp,response-error-message,city,cityCode,countryCode,...,gepirParty-partyDataLine-gepirRequestedKey,gepirParty-partyDataLine-informationProvider,gepirParty-partyDataLine-lastChangeDate,gepirParty-partyDataLine-partyDataLanguage,gepirParty-partyDataLine-responderSpecificData,gepirParty-partyDataLine-returnCode,responseHeader-gepirReturnCode-_,responseHeader-numberOfHits,responseHeader-responderGLN,status
0,60409075000152,2018-10-18 18:32:57,07891000258491,b6aee075-6489-4bb3-abe9-5da6cee3cf71,None,None,None,SAO PAULO,,"(, 1)",...,"((GTIN, None), 07891000258491, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK
1,54360656001701,2019-01-19 12:03:31,07898142863118,a49e7c08-6788-48bf-8061-d24f48374260,None,None,None,CAMPINAS,,"(, 1)",...,"((GTIN, None), 07898142863118, None)","(7890000000000, GS1 Brazil, None)",None,"(, 1)","(, )","(0, 1)",0,1,9501101020016,OK


In [164]:
# Visualizando o schema do DataFrame
gs7.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response-error-code: string (nullable = true)
 |-- response-error-extendedHelp: string (nullable = true)
 |-- response-error-message: string (nullable = true)
 |-- city: string (nullable = true)
 |-- cityCode: string (nullable = true)
 |-- countryCode: struct (nullable = true)
 |    |-- _: string (nullable = true)
 |    |-- codeListVersion: string (nullable = true)
 |-- countyCode: string (nullable = true)
 |-- crossStreet: string (nullable = true)
 |-- currencyOfPartyCode: struct (nullable = true)
 |    |-- _: string (nullable = true)
 |    |-- codeListVersion: string (nullable = true)
 |-- geographicalCoordinates: struct (nullable = true)
 |    |-- latitude: string (nullable = true)
 |    |-- longitude: string (nullable = true)
 |-- languageOfThePartyCode: struct (nullable = true)
 |    |-- _: string (nullable = tru

In [165]:
# Renomeando as colunas expandidas com o alias da coluna anterior

gs8 =( gs7.select(
        gs7['cnpj_manufacturer'], 
        gs7['created_at'], 
        gs7['gtin'], 
        gs7['id'],
        gs7['response-error-code'],
        gs7['response-error-extendedHelp'],
        gs7['response-error-message'],
        col('city').alias('gepirParty-partyDataLine-address-city'),
        col('cityCode').alias('gepirParty-partyDataLine-address-cityCode'),
        col('crossStreet').alias('gepirParty-partyDataLine-address-crossStreet'),
        col('name').alias('gepirParty-partyDataLine-address-name'),
        col('pOBoxNumber').alias('gepirParty-partyDataLine-address-pOBoxNumber'),
        col('postalCode').alias('gepirParty-partyDataLine-address-postalCode'),
        col('provinceCode').alias('gepirParty-partyDataLine-address-provinceCode'),
        col('state').alias('gepirParty-partyDataLine-address-state'),
        col('streetAddressOne').alias('gepirParty-partyDataLine-address-streetAddressOne'),
        col('streetAddressThree').alias('gepirParty-partyDataLine-address-streetAddressThree'),
        col('postalCode').alias('gepirParty-partyDataLine-address-postalCode'),
        gs7['gepirParty-partyDataLine-countryAdministered'],
        gs7['gepirParty-partyDataLine-gS1CompanyPrefix'],
        gs7['gepirParty-partyDataLine-gepirItemExternalFileLink'],
        gs7['gepirParty-partyDataLine-lastChangeDate'],
        gs7['responseHeader-gepirReturnCode-_'],
        gs7['responseHeader-numberOfHits'],
        gs7['responseHeader-responderGLN'],
        gs7['status'])
        )

In [166]:
gs8.printSchema()

root
 |-- cnpj_manufacturer: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- gtin: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response-error-code: string (nullable = true)
 |-- response-error-extendedHelp: string (nullable = true)
 |-- response-error-message: string (nullable = true)
 |-- gepirParty-partyDataLine-address-city: string (nullable = true)
 |-- gepirParty-partyDataLine-address-cityCode: string (nullable = true)
 |-- gepirParty-partyDataLine-address-crossStreet: string (nullable = true)
 |-- gepirParty-partyDataLine-address-name: string (nullable = true)
 |-- gepirParty-partyDataLine-address-pOBoxNumber: string (nullable = true)
 |-- gepirParty-partyDataLine-address-postalCode: string (nullable = true)
 |-- gepirParty-partyDataLine-address-provinceCode: string (nullable = true)
 |-- gepirParty-partyDataLine-address-state: string (nullable = true)
 |-- gepirParty-partyDataLine-address-streetAddressOne: string (nullable = true)
 |--

In [167]:
df_gs1 = (gs8.select(
            col('cnpj_manufacturer').alias('cnpj_manufacturer'),
            col('gtin').alias('gtin'),
            col('gepirParty-partyDataLine-address-city').alias('city'),
            col('gepirParty-partyDataLine-address-state').alias('state'),
            col('gepirParty-partyDataLine-address-name').alias('name'),
            col('status').alias('status'))
)

In [168]:
df_gs1.limit(5).toPandas()

,cnpj_manufacturer,gtin,city,state,name,status
0,60409075000152,07891000258491,SAO PAULO,SP,NESTLE BRASIL LTDA,OK
1,54360656001701,07898142863118,CAMPINAS,SP,ARCOR DO BRASIL LTDA BRAGANCA PAULISTA,OK
2,14998371000119,07896005213018,FORTALEZA,CE,J MACEDO S.A.,OK
3,None,08008245003420,Govone,None,Nutkao Srl,OK
4,None,07898043230798,None,None,None,OK


In [169]:
# Lendo o arquivo cnpjs_receita_federal.jl e inserindo em um DataFrame
cnpjs = spark.read.format('json').load('Desktop\data_etl_test (1)\cnpjs_receita_federal.jl',\
                                                       multiLine = "False")

In [170]:
cnpjs.limit(2).toPandas()

,cnpj,created_at,id,response
0,68093095000179,2018-10-19 21:36:11,99068e55-1018-47e7-91a9-44cded045033,"(01/07/1992, [(10.31-7-00, Fabricação de conse..."
1,13492669000190,2018-10-20 17:17:03,c475b934-eef1-488e-8f6a-b6087ef32b9b,"(17/03/2011, [(46.35-4-03, Comércio atacadista..."


In [171]:
cnpjs.printSchema()

root
 |-- cnpj: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response: struct (nullable = true)
 |    |-- abertura: string (nullable = true)
 |    |-- atividade_principal: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- atividades_secundarias: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- billing: struct (nullable = true)
 |    |    |-- database: boolean (nullable = true)
 |    |    |-- free: boolean (nullable = true)
 |    |-- capital_social: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cnpj: string (nullable = true)
 |    |-- complemento: string (nullable = true)
 |    |-- 

In [172]:
cnpjs2 = (cnpjs.select(
          col('cnpj').alias('cnpj1'),
          cnpjs['created_at'],
          cnpjs['id'],
          col('response.*'))
)

In [173]:
cnpjs2.limit(2).toPandas()

,cnpj1,created_at,id,abertura,atividade_principal,atividades_secundarias,bairro,billing,capital_social,cep,...,numero,porte,qsa,situacao,situacao_especial,status,telefone,tipo,uf,ultima_atualizacao
0,68093095000179,2018-10-19 21:36:11,99068e55-1018-47e7-91a9-44cded045033,01/07/1992,"[(10.31-7-00, Fabricação de conservas de frutas)]","[(10.53-8-00, Fabricação de sorvetes e outros ...",IARA,"(True, True)",39950000.00,12.949-880,...,S/N,None,"[(MARCELO CESANA, None, None, 10-Diretor, None...",ATIVA,,OK,(11) 3093-3100 / (11) 3093-1300,MATRIZ,SP,2018-08-09T18:58:13.133Z
1,13492669000190,2018-10-20 17:17:03,c475b934-eef1-488e-8f6a-b6087ef32b9b,17/03/2011,"[(46.35-4-03, Comércio atacadista de bebidas c...","[(46.39-7-01, Comércio atacadista de produtos ...",PINHEIROS,"(True, True)",76729078.00,05.407-000,...,359,None,"[(HIJOS DE RIVERA SA, FABIO RODRIGUES, ESPANHA...",ATIVA,,OK,(11) 2368-5876,MATRIZ,SP,2018-09-05T15:13:52.878Z


In [174]:
cnpjs2.printSchema()

root
 |-- cnpj1: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- abertura: string (nullable = true)
 |-- atividade_principal: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- atividades_secundarias: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- billing: struct (nullable = true)
 |    |-- database: boolean (nullable = true)
 |    |-- free: boolean (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- data_situacao: string (nullable = true)
 |-- data_situacao_especial: string (nullable = true)
 |-- efr: string (nullable = true)


In [175]:
df_cnpjs = (cnpjs2.select(
          col('cnpj1').alias('cnpj_somente_digitos'),
          cnpjs2['nome'],
          cnpjs2['fantasia'],
          cnpjs2['bairro'],
          cnpjs2['municipio'],
          cnpjs2['uf'],
          cnpjs2['cnpj'])
)

In [176]:
df_cnpjs.toPandas()

,cnpj_somente_digitos,nome,fantasia,bairro,municipio,uf,cnpj
0,68093095000179,FROOTY COMERCIO E INDUSTRIA DE ALIMENTOS S.A.,"SP ICE, FROOTY ACAI, ACAI FROOTY",IARA,ATIBAIA,SP,68.093.095/0001-79
1,13492669000190,ESTRELLA DE GALICIA IMPORTACAO E COMERCIALIZAC...,,PINHEIROS,SAO PAULO,SP,13.492.669/0001-90
2,74683392000177,DROGARIA NOVA SABARA LTDA,,JARDIM ELLUS,SAO PAULO,SP,74.683.392/0001-77
3,12423658000195,DJ INDUSTRIA E COMERCIO DE ALIMENTOS LTDA,PRAEIRA ALIMENTOS,SAO SEBASTIAO,CARPINA,PE,12.423.658/0001-95
4,00588458000103,FUGINI ALIMENTOS LTDA,,JARDIM PARAISO,MONTE ALTO,SP,00.588.458/0001-03
...,...,...,...,...,...,...,...
989,72861461000160,CAFE PACAEMBU LTDA,CAFE PACAEMBU,JARDIM SANTA MARTA,VARGEM GRANDE DO SUL,SP,72.861.461/0001-60
990,60945169001037,NISSIN FOODS DO BRASIL LTDA,,VOTORANTIM,IBIUNA,SP,60.945.169/0010-37
991,71669766000101,FARMACIA MAJESTIC EIRELI,YOFARMA FORMULAS ESPECIALIZADAS,CENTRO,SAO ROQUE,SP,71.669.766/0001-01
992,03870455000156,FORNO DE MINAS ALIMENTOS S/A,,ZONA RURAL,CONCEICAO DO PARA,MG,03.870.455/0001-56


In [177]:
# DataFrames finais
# df_infomix
# df_gs1
# df_cnpjs
# df_cosmos
# df_descricoes_externas

# Juntando os DataFrames

## Utilizando SparkSQL

In [178]:
df_infomix.limit(5).toPandas()

,cnpj,gtin,category
0,08540603000198,06890101502534,massas e molhos
1,71322150004408,07898605250417,cervejas
2,43235985000732,07896000724465,cabelo
3,47508411134004,07896102589122,conservas e enlatados
4,47508411135833,07895000474646,bomboniere


In [179]:
df_infomix.count()

25844

In [180]:
df_infomix.createOrReplaceTempView('tb_infomix')

In [181]:
spark.sql('SELECT * FROM tb_infomix').limit(5).toPandas()

,cnpj,gtin,category
0,08540603000198,06890101502534,massas e molhos
1,71322150004408,07898605250417,cervejas
2,43235985000732,07896000724465,cabelo
3,47508411134004,07896102589122,conservas e enlatados
4,47508411135833,07895000474646,bomboniere


In [182]:
df_gs1.limit(5).toPandas()

,cnpj_manufacturer,gtin,city,state,name,status
0,60409075000152,07891000258491,SAO PAULO,SP,NESTLE BRASIL LTDA,OK
1,54360656001701,07898142863118,CAMPINAS,SP,ARCOR DO BRASIL LTDA BRAGANCA PAULISTA,OK
2,14998371000119,07896005213018,FORTALEZA,CE,J MACEDO S.A.,OK
3,None,08008245003420,Govone,None,Nutkao Srl,OK
4,None,07898043230798,None,None,None,OK


In [183]:
df_gs1.createOrReplaceTempView('tb_gs1')

In [184]:
spark.sql('SELECT * FROM tb_gs1').limit(5).toPandas()

,cnpj_manufacturer,gtin,city,state,name,status
0,60409075000152,07891000258491,SAO PAULO,SP,NESTLE BRASIL LTDA,OK
1,54360656001701,07898142863118,CAMPINAS,SP,ARCOR DO BRASIL LTDA BRAGANCA PAULISTA,OK
2,14998371000119,07896005213018,FORTALEZA,CE,J MACEDO S.A.,OK
3,None,08008245003420,Govone,None,Nutkao Srl,OK
4,None,07898043230798,None,None,None,OK


In [185]:
df_cnpjs.createOrReplaceTempView('tb_cnpjs')

In [186]:
spark.sql('SELECT * FROM tb_cnpjs').limit(5).toPandas()

,cnpj_somente_digitos,nome,fantasia,bairro,municipio,uf,cnpj
0,68093095000179,FROOTY COMERCIO E INDUSTRIA DE ALIMENTOS S.A.,"SP ICE, FROOTY ACAI, ACAI FROOTY",IARA,ATIBAIA,SP,68.093.095/0001-79
1,13492669000190,ESTRELLA DE GALICIA IMPORTACAO E COMERCIALIZAC...,,PINHEIROS,SAO PAULO,SP,13.492.669/0001-90
2,74683392000177,DROGARIA NOVA SABARA LTDA,,JARDIM ELLUS,SAO PAULO,SP,74.683.392/0001-77
3,12423658000195,DJ INDUSTRIA E COMERCIO DE ALIMENTOS LTDA,PRAEIRA ALIMENTOS,SAO SEBASTIAO,CARPINA,PE,12.423.658/0001-95
4,00588458000103,FUGINI ALIMENTOS LTDA,,JARDIM PARAISO,MONTE ALTO,SP,00.588.458/0001-03


In [187]:
df_cosmos.createOrReplaceTempView('tb_cosmos')

In [188]:
spark.sql('SELECT * FROM tb_cosmos').limit(5).toPandas()

,gtin,gtin2,brand,description,category_description,status
0,07896040700276,07896040700276,None,LAVA ROUPAS COQUETEL ERVAS,Detergentes para a roupa,OK
1,00000078909434,00000078909434,FERRERO ROCHER,BOMBOM FERRERO ROCHER 3X16G,None,OK
2,07898917945100,None,None,None,None,GTIN_NOT_FOUND
3,27702018072396,None,None,None,None,GTIN_NOT_FOUND
4,07896005279595,None,None,None,None,GTIN_NOT_FOUND


In [189]:
df_descricoes_externas.createOrReplaceTempView('tb_descricoes_externas')

In [190]:
spark.sql('SELECT * FROM tb_descricoes_externas').limit(5).toPandas()

,id,gtin,description,flag_infoprice
0,2482686,07896512911933,SABONETE INFANTIL LIQUIDO GRANADO BEBE LAVANDA...,true
1,2482685,07896512911933,SAB GRANADO BEBE LAVANDA 250G,false
2,2482684,07896512911933,SAB LIQ.GRAN.BEBE LAVAN 25,false
3,2482683,07896512911933,SAB GRANADO GLICLAVAN BEBE 250M,false
4,2482682,07896512911933,SAB BABY GRA LA 250ML,false


In [191]:
df_final = spark.sql("""SELECT 
             tb_infomix.gtin as gtin_infomix,
             tb_gs1.cnpj_manufacturer as cnpj,
             tb_gs1.name as razao_social,
             tb_gs1.city as cidade,
             tb_gs1.state as estado,
             tb_cnpjs.nome as razao_social_cnpj,
             tb_cnpjs.municipio as cidade_cnpj,
             tb_cnpjs.uf as estado_cnpj,
             tb_cosmos.description as descricao,
             tb_cosmos.category_description as categoria
             from tb_infomix
             left join tb_gs1 
             on (tb_infomix.gtin == tb_gs1.gtin)
             left join tb_cnpjs
             on (tb_infomix.cnpj == tb_cnpjs.cnpj_somente_digitos)
             left join tb_cosmos
             on (tb_infomix.gtin == tb_cosmos.gtin)
             """)

In [192]:
df_final.limit(5).toPandas()

,gtin_infomix,cnpj,razao_social,cidade,estado,razao_social_cnpj,cidade_cnpj,estado_cnpj,descricao,categoria
0,06890101502534,None,None,None,None,SUPERMERCADO LESTE OESTE LTDA,MACEIO,AL,None,None
1,07898605250417,01131570000183,CERVEJARIA ZX S.A.,JAGUARIUNA,SP,SAVEGNAGO-SUPERMERCADOS LTDA,SAO CARLOS,SP,None,None
2,07898605250417,01131570000183,CERVEJARIA ZX S.A.,JAGUARIUNA,SP,SAVEGNAGO-SUPERMERCADOS LTDA,SAO CARLOS,SP,None,None
3,07898605250417,01131570000183,CERVEJARIA ZX S.A.,JAGUARIUNA,SP,SAVEGNAGO-SUPERMERCADOS LTDA,SAO CARLOS,SP,None,None
4,07896000724465,30153506000100,Niely do Brasil Industrial Ltda,NOVA IGUACU,RJ,CATRICALA E CIA LIMITADA,SAO JOSE DO RIO PRETO,SP,CREME TRATAMENTO NIELY GOLD 430G MATIZADOR UNIT,None


In [193]:
spark.stop()